In [416]:
# Import enough libraries for initialize de dataset.
import pandas as pd
import re

# Load the data from the csv file.
df = pd.read_csv('survey_results_public.csv')

# How many rows and columns we have
print("Rows:", df.shape[0])
print("Columns:", df.shape[1])

Número de filas: 73268
Número de columnas: 79


In [417]:
# We define a function to handle specially cases like age, because we don't have a specific age by every record. So, will try to make a prom by every record based on the min value and max value.
def extract_age(text):
    pattern = r'(\d+)-(\d+) years old'
    match = re.match(pattern, str(text))
    if match:
        start_age = int(match.group(1))
        end_age = int(match.group(2))
        return (start_age + end_age) // 2
    return None

# We apply the same procedure to handle how many people there are in the organization where developer works.
def extract_num_of_employees(text):
    pattern = r'(\d+(?:,\d+)?)\s?to\s?(\d+(?:,\d+)?)'
    match = re.match(pattern, text)
    if match:
        start_value = int(match.group(1).replace(',', ''))
        end_value = int(match.group(2).replace(',', ''))
        return (start_value + end_value) // 2
    return None

pd.set_option('display.float_format', '{:.0f}'.format)

In [418]:
# Trying to delete duplicated records.
df = df.drop_duplicates()

# We set a list of columns we want to process.
desired_columns = ['Employment', 'Age', 'Gender', 'Ethnicity', 'RemoteWork', 'CodingActivities', 'EdLevel', 'YearsCode', 'YearsCodePro', 'DevType', 'OrgSize', 'Country', 'ConvertedCompYearly', 'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 'PlatformHaveWorkedWith','WebframeHaveWorkedWith', 'OpSysProfessional use']

# We reset old data with new no segregated data
df = df.loc[:, desired_columns]

# We show some records we have, only the first five ones
print(df.head(5))

            Employment              Age Gender Ethnicity  \
0                  NaN              NaN    NaN       NaN   
1  Employed, full-time              NaN    NaN       NaN   
2  Employed, full-time  25-34 years old    Man     White   
3  Employed, full-time  35-44 years old    Man     White   
4  Employed, full-time  25-34 years old    NaN       NaN   

                             RemoteWork  \
0                                   NaN   
1                          Fully remote   
2  Hybrid (some remote, some in-person)   
3                          Fully remote   
4  Hybrid (some remote, some in-person)   

                           CodingActivities  \
0                                       NaN   
1  Hobby;Contribute to open-source projects   
2                                     Hobby   
3              I don’t code outside of work   
4                                     Hobby   

                                           EdLevel YearsCode YearsCodePro  \
0                   

In [419]:
# Because we don't know all the values are correct to be processed, we look at the unique values for that column to check it.
datos_agrupados = df['Age'].unique()

print(datos_agrupados)

[nan '25-34 years old' '35-44 years old' 'Under 18 years old'
 '18-24 years old' '45-54 years old' '55-64 years old' '65 years or older'
 'Prefer not to say']


In [420]:
# Trying to clean and prepare the data to be processed, setting it in a right format that function accepts.
df['Age'] = df['Age'].replace(
    [pd.NaT, 'Under 18 years old', '65 years or older', 'Prefer not to say'],
    ['0-0 years old', '17-17 years old', '65-65 years old', '0-0 years old'])

# We ensure about the changes.
datos_agrupados = df['Age'].unique()

print(datos_agrupados)

['0-0 years old' '25-34 years old' '35-44 years old' '17-17 years old'
 '18-24 years old' '45-54 years old' '55-64 years old' '65-65 years old']


In [421]:
# Son we apply the function to extract the age
df['Age'] = df['Age'].apply(extract_age)

# We convert the age to int value
df['Age'] = df['Age'].astype(int)

# We ensure about the changes and print it
datos_agrupados = df['Age'].unique()

print(datos_agrupados)

[ 0 29 39 17 21 49 59 65]


In [422]:
# We apply the same process for the column OrgSize.
df['OrgSize'] = df['OrgSize'].replace(
    [pd.NaT, 'I don’t know', 'Just me - I am a freelancer, sole proprietor, etc.','10,000 or more employees'],
    ['0 to 0 employees', '0 to 0 employees', '1 to 1 employees','10,000 to 10,000 employees'])

# Aplicar la función de extracción de empleados a la columna 'OrgSize'
df['OrgSize'] = df['OrgSize'].apply(extract_num_of_employees)

datos_agrupados = df['OrgSize'].unique()

print(datos_agrupados)

[    0    59   299     1     5  7499  2999 10000   749    14]


In [423]:
# We select only the integer ones columns in the dataset, and storage it in a new var.
# We want to work only with this data to calculate values as mean, median and mode
data_numeric = df.select_dtypes(include='number')

media = data_numeric.mean(numeric_only=True)

# Cleaning and setting right values for YearsCode
df['YearsCode'] = df['YearsCode'].replace(
    [pd.NaT, 'More than 50 years', 'Less than 1 year'],
    ['0', '0', '0'])

df['YearsCode'] = df['YearsCode'].astype(int)

# Cleaning and setting right values for YearsCodePro
df['YearsCodePro'] = df['YearsCodePro'].replace(
    [pd.NaT, 'More than 50 years', 'Less than 1 year'],
    ['0', '0', '0'])

df['YearsCodePro'] = df['YearsCodePro'].astype(int)

data_numeric = df.select_dtypes(include='number')

# Finally we print the media data
media = data_numeric.mean(numeric_only=True)

print(media)



Age                       30
YearsCode                 12
YearsCodePro               7
OrgSize                 1488
ConvertedCompYearly   170761
dtype: float64


In [424]:
# Finally we print the median data
media = data_numeric.median()

print(media)

Age                      29
YearsCode                 9
YearsCodePro              4
OrgSize                  59
ConvertedCompYearly   67845
dtype: float64


In [425]:
# Finally we print the mode data
media = data_numeric.mode()

print(media)

   Age  YearsCode  YearsCodePro  OrgSize  ConvertedCompYearly
0   29         10             0        0               150000


In [426]:
# Finally we save all the modified data in a new csv file.
df.to_csv('dataset1_cleaned.csv', index=False)